In [1]:
# https://towardsdatascience.com/pandas-groupby-a-simple-but-detailed-tutorial-314b8f37005d
# https://towardsdatascience.com/accessing-data-in-a-multiindex-dataframe-in-pandas-569e8767201d
# https://towardsdatascience.com/summarizing-data-with-pandas-crosstab-efc8b9abecf
# https://towardsdatascience.com/how-to-flatten-multiindex-columns-and-rows-in-pandas-f5406c50e569
# https://datascientyst.com/list-aggregation-functions-aggfunc-groupby-pandas/
# https://stackoverflow.com/questions/25929319/how-to-iterate-over-pandas-multiindex-dataframe-using-index
# https://stackoverflow.com/questions/24495695/pandas-get-unique-multiindex-level-values-by-label
# https://stackoverflow.com/questions/55706391/pandas-crosstab-on-multiple-columns-then-groupby

# https://matplotlib.org/stable/gallery/pyplots/pyplot_text.html#sphx-glr-gallery-pyplots-pyplot-text-py

In [2]:
import pandas as pd
import numpy as np
from myUtils import pickle_load, pickle_dump

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

# # filename_symbols = path_data_dump + 'vg_symbols_4chars_max.csv'  # symbols text file
# filename_symbols = path_data_dump + 'my_symbols.csv'  # symbols text file

# _filename_pickled_df_OHLCVA_downloaded = 'df_OHLCVA_downloaded '  # OHLCVA downloaded from Yahoo
filename_pickled_df_adjOHLCV = 'df_adjOHLCV'  # adjusted OHLCV
filename_pickled_df_symbols_close = "df_symbols_close"  # symbols' adjusted close
filename_pickled_symbols_df_adjOHLCV =  'symbols_df_adjOHLCV'  # symbols in df_adjOHLCV
filename_pickled_perf_rank_dict =  'perf_rank_dict'  # store symbols from performance rank results
filename_pickled_r_all_ranks =  'r_all_ranks'  # list of top 100 most common symbols from performance rank results
filename_pickled_df_a = 'df_OHLCV_clean'  # df adjusted OHLCV, dropped symbols with no vol and close
filename_pickled_df_c = 'df_close_clean'  # df close, dropped symbols with no vol and close

verbose = False  # True prints more output

#################
# look_back_days = -250 * 60  # subset df iloc days
look_back_days = -250 * 6  # subset df iloc days, 6 years of data
#################

In [3]:
print(f"Full path to pickled df_symbols_close:  {path_data_dump}{filename_pickled_df_symbols_close}")
df_close = pickle_load(path_data_dump, filename_pickled_df_symbols_close, verbose=verbose)
print(f"Full path to pickled df_adjOHLCV:  {path_data_dump}{filename_pickled_df_adjOHLCV}")
df_adjOHLCV = pickle_load(path_data_dump, filename_pickled_df_adjOHLCV, verbose=verbose)

Full path to pickled df_symbols_close:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_symbols_close
Full path to pickled df_adjOHLCV:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_adjOHLCV


In [4]:
# https://stackoverflow.com/questions/63826291/pandas-series-find-column-by-value
df = df_adjOHLCV[look_back_days::]
df_v = df.xs('Volume', level=1, axis=1)  # select only Volume columns
rows, cols = np.where(df_v == 0)  # row index, column index where trading volumes are zero
idx_no_volume = list(set(cols))
idx_no_volume.sort()
symbols_no_volume = df_v.columns[idx_no_volume]
print(f'symbols with no volume:\n{symbols_no_volume}')

symbols with no volume:
Index(['AMCR', 'AVT', 'BIIB', 'BKR', 'CCI', 'CHD', 'CNC', 'CNXC', 'DISH',
       'ESAB', 'FCFS', 'GEN', 'IBKR', 'JJC', 'NSTG', 'PEAK', 'PFG', 'SBNY',
       'SRPT', 'SSB', 'VC', 'VNT', 'WH', 'XEL'],
      dtype='object')


In [5]:
df_dif = df_v - df_v.shift(periods=1)
rows, cols = np.where(df_dif == 0)
idx_same_volume = list(set(cols))
idx_same_volume.sort()
idx_same_volume
symbols_same_volume = df_v.columns[idx_same_volume]
print(f'symbols with same volume:\n{symbols_same_volume}')

symbols with same volume:
Index(['ABM', 'ACM', 'ACN', 'AGCO', 'ALE', 'ALRM', 'AMCR', 'AMED', 'AMG',
       'AMGN',
       ...
       'VRSK', 'VSAT', 'WDAY', 'WERN', 'WEX', 'WSFS', 'WTS', 'WTW', 'XEL',
       'XENE'],
      dtype='object', length=253)


In [6]:
df_c = df.xs('Close', level=1, axis=1)  # select only Close columns
df_c = df_c.fillna(0).copy()  # convert NaNs to zeros
rows, cols = np.where(df_c == 0)  # row index, column index where trading volumes are zero
idx_no_close = list(set(cols))
idx_no_close.sort()
symbols_no_close = df_c.columns[idx_no_close]
print(f'symbols with NaN close:\n{symbols_no_close}')

symbols with NaN close:
Index(['ABNB', 'AIRC', 'ALGM', 'BJ', 'BRBR', 'BTC-USD', 'BVH', 'CARR', 'CDAY',
       'CEG', 'CHK', 'CHX', 'CNXC', 'CR', 'CRWD', 'CTVA', 'DDOG', 'DISH',
       'DOW', 'DT', 'DTM', 'ESAB', 'ETH-USD', 'ETRN', 'FOX', 'FOXA', 'FYBR',
       'GEHC', 'GFS', 'GO', 'GXO', 'JJC', 'KD', 'LCID', 'MP', 'MRNA', 'NARI',
       'NSTG', 'NVST', 'NVT', 'OGN', 'OTIS', 'PDD', 'PGNY', 'RIVN', 'SEAS',
       'SGOV', 'SHC', 'SITM', 'SRC', 'SWAV', 'VMW', 'VNT', 'VSCO', 'WH',
       'YETI', 'ZM', 'ZS'],
      dtype='object')


In [7]:
symbols_bad_data = list(symbols_no_close) + list(symbols_no_volume) + list(symbols_same_volume) # combine symbols with no volume and no close
unique_symbols_bad_data = sorted(list(set(symbols_bad_data)))  # unique symbols
print(f'unique symbols with bad data, e.g. no volume, same volume and $0 close, includes duplicate symbols: {len(unique_symbols_bad_data)}')

unique symbols with bad data, e.g. no volume, same volume and $0 close, includes duplicate symbols: 315


In [8]:
# get symbols of past model picks
df_picks = pickle_load(path_data_dump, 'df_picks', verbose=verbose)
df_picks.head(1)

,date_end_df_train,max_days_lookbacks,days_lookbacks,sym_freq_15,sym_freq_14,sym_freq_13,sym_freq_12,sym_freq_11,sym_freq_10,sym_freq_9,sym_freq_8,sym_freq_7,sym_freq_6,sym_freq_5,sym_freq_4,sym_freq_3,sym_freq_2
0,2024-02-15,120,"[30, 60, 120]",[],[],[],[],[],[],[],[],"['SHV', 'SMCI']",['ANF'],"['AGO', 'FATE', 'FTSM', 'META', 'NTNX', 'WING']","['CLS', 'COHR', 'ICHR', 'PGR']","['COST', 'DOOR', 'EXP', 'GBTC', 'GDDY', 'HA', ...",[]


In [9]:
# keep only columns with symbols of past picks
df_picks.drop(columns=["date_end_df_train", "max_days_lookbacks", "days_lookbacks"], inplace=True)
df_picks.head(1)

,sym_freq_15,sym_freq_14,sym_freq_13,sym_freq_12,sym_freq_11,sym_freq_10,sym_freq_9,sym_freq_8,sym_freq_7,sym_freq_6,sym_freq_5,sym_freq_4,sym_freq_3,sym_freq_2
0,[],[],[],[],[],[],[],[],"['SHV', 'SMCI']",['ANF'],"['AGO', 'FATE', 'FTSM', 'META', 'NTNX', 'WING']","['CLS', 'COHR', 'ICHR', 'PGR']","['COST', 'DOOR', 'EXP', 'GBTC', 'GDDY', 'HA', ...",[]


In [10]:
df_picks.columns

Index(['sym_freq_15', 'sym_freq_14', 'sym_freq_13', 'sym_freq_12',
       'sym_freq_11', 'sym_freq_10', 'sym_freq_9', 'sym_freq_8', 'sym_freq_7',
       'sym_freq_6', 'sym_freq_5', 'sym_freq_4', 'sym_freq_3', 'sym_freq_2'],
      dtype='object')

In [11]:
import ast

In [12]:
list_of_lists = []

for col in df_picks.columns:
  # convert column values from string to list, e.g. '[]', '[]', '["A", "B", ..]' ... to [], [], ["A", "B", ..], ...
  l_series = df_picks[col].apply(ast.literal_eval)
  # list_of_lists = [l_item for l_item in l_series if l_item]  # this doesn't works  
  for l_item in l_series:
      if l_item:  # 
        list_of_lists.append(l_item)  

symbols_picks = [val for sublist in list_of_lists for val in sublist]
print(f'symbol count from model picks: {len(symbols_picks)}')
# list sorted unique symbols
unique_symbols_picks = sorted(list(set(symbols_picks)))
print(f'unique symbol count from model picks: {len(unique_symbols_picks)}')

symbol count from model picks: 56640
unique symbol count from model picks: 1005


In [13]:
def find_common_items(list1, list2):
  """Finds the common items between two lists.

  Args:
    list1: The first list.
    list2: The second list.

  Returns:
    A list of the common items between the two lists.
  """

  common_items = []
  for item in list1:
    if item in list2:
      common_items.append(item)
  return common_items


# code in if block runs only from command line, code will NOT be executed if imported as a module
if __name__ == "__main__": 
  list1 = ["a", "b", "c", "d", "e"]
  list2 = ["b", "c", "d", "f", "g"]
  common_items = find_common_items(list1, list2)
  print(common_items)

['b', 'c', 'd']


In [14]:
# symbols are in past picks but also have bad data 
common_symbols = find_common_items(unique_symbols_picks, unique_symbols_bad_data)
print(common_symbols)

['ALE', 'ARCH', 'AZPN', 'BTC-USD', 'CHT', 'CMG', 'CVBF', 'CVLT', 'DISH', 'DVA', 'ETH-USD', 'HOLI', 'HY', 'LOB', 'MLKN', 'MLR', 'NSTG', 'OSIS', 'PKX', 'PTMN', 'RRC', 'SEAS', 'SFBS', 'SRPT', 'TRGP', 'TX', 'VNO', 'WDAY', 'XENE']


In [15]:
def subtract_items(list1, list2):
  """Subtracts items in list2 from items in list1.

  Args:
    list1: The first list.
    list2: The second list.

  Returns:
    A list of the items in list1 that are not in list2.
  """

  subtracted_items = []
  for item in list1:
    if item not in list2:
      subtracted_items.append(item)
  return subtracted_items

In [16]:
symbols_drop = subtract_items(unique_symbols_bad_data, common_symbols)  # don't drop symbols in past picks
symbols_drop .sort()
print(f'len(unique_symbols_bad_data): {len(unique_symbols_bad_data)}')
print(f'len(common_symbols): {len(common_symbols)}')
print(f'len(symbols_drop): {len(symbols_drop)}')

len(unique_symbols_bad_data): 315
len(common_symbols): 29
len(symbols_drop): 286


In [17]:
# symbols_drop = list(symbols_no_close) + list(symbols_no_volume) + list(symbols_same_volume) # combine symbols with no volume and no close
# print(f'combined symbols with no volume, same volume and no close, inculdes duplicate symbols: {len(symbols_drop)}')
# symbols_drop = list(set(symbols_drop))  # drop duplicate symbols
# symbols_drop .sort()


df_a = df.drop(symbols_drop, axis=1, level=0)  # drop symbols from OHLCA df
df_c = df_close.iloc[look_back_days::]
df_c = df_c.drop(symbols_drop, axis=1)
print(f'unique symbols dropped from df_a (adjOLHLV) and df_c (Close): {len(symbols_drop)}')

unique symbols dropped from df_a (adjOLHLV) and df_c (Close): 286


In [18]:
print(f'symbols with no volume:   {len(symbols_no_volume):>5,}')
print(f'symbols with same volume: {len(symbols_same_volume):>5,}')
print(f'symbols with no close:    {len(symbols_no_close):>5,}\n')
print(f'symbols total before drop:                                        {len(df_close.columns):>5,}')
print(f'unique symbols dropped from df OHLCVA (df_a) and df Close (df_c): {len(symbols_drop):>5,}\n')
print('                                          symbols     rows')
print(f'df adjOHLCV (df_a) after dropped symbols:   {len(df_a.columns)/5:>5,.0f}    {len(df_a):>5,}')
print(f'df Close (df_c) after dropped symbols:      {len(df_c.columns):>5,}    {len(df_c):>5,}')


symbols with no volume:      24
symbols with same volume:   253
symbols with no close:       58

symbols total before drop:                                        1,548
unique symbols dropped from df OHLCVA (df_a) and df Close (df_c):   286

                                          symbols     rows
df adjOHLCV (df_a) after dropped symbols:   1,262    1,500
df Close (df_c) after dropped symbols:      1,262    1,500


In [19]:
pickle_dump(df_a, path_data_dump, filename_pickled_df_a)
print(f'pickled df adjOHLCV after dropping symbols with no volume, same volume, and no close:\n{path_data_dump}{filename_pickled_df_a}')
pickle_dump(df_c, path_data_dump, filename_pickled_df_c)
print(f'pickled df Close after dropping symbols with no volume, same volume, and no close:\n{path_data_dump}{filename_pickled_df_c}')

pickled df adjOHLCV after dropping symbols with no volume, same volume, and no close:
C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_OHLCV_clean
pickled df Close after dropping symbols with no volume, same volume, and no close:
C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_close_clean


In [20]:
from myUtils import list_dump

f_symbols_df_close_clean = 'symbols_df_close_clean.csv'  # symbols text file
symbols_df_c = list(df_c)  # column names in df_c
list_dump(symbols_df_c, path_data_dump, f_symbols_df_close_clean)# df_c.columns.to_csv(f_symbols_df_close_clean)

In [21]:
df_a.tail()

A                                                 \
                  Open        High         Low       Close     Volume   
Date                                                                    
2024-02-12  133.279999  133.279999  130.380005  132.309998  1231800.0   
2024-02-13  130.339996  131.639999  128.259995  129.759995  1031100.0   
2024-02-14  131.000000  132.330002  130.210007  132.240005  1405200.0   
2024-02-15  132.990005  135.600006  132.679993  134.750000  1514500.0   
2024-02-16  133.589996  136.270004  133.589996  134.839996  1035790.0   

                   AA                                              ...  \
                 Open       High        Low      Close     Volume  ...   
Date                                                               ...   
2024-02-12  27.100000  28.094999  26.870001  27.900000  4374600.0  ...   
2024-02-13  26.900000  27.200001  25.575001  25.850000  7264200.0  ...   
2024-02-14  26.209999  26.990000  25.870001  26.790001  5100600.0  ...   
2024-02-15  27.040001  27.639999  26.920000  27.389999  4385500.0  ...   
2024-02-16  27.340000  28.030001  27.160000  27.400000  4382574.0  ...   

                 ZUMZ                                                   ZWS  \
                 Open       High        Low      Close    Volume       Open   
Date                                                                          
2024-02-12  18.450001  19.260000  18.450001  19.000000  163900.0  32.209999   
2024-02-13  18.120001  18.270000  17.670000  17.870001  182700.0  31.230000   
2024-02-14  18.230000  18.230000  17.639999  18.000000   89000.0  31.360001   
2024-02-15  18.170000  18.559999  18.040001  18.480000  135000.0  31.799999   
2024-02-16  18.240000  18.549999  18.190001  18.309999   89355.0  31.750000   

                                                        
                 High        Low      Close     Volume  
Date                                                    
2024-02-12  32.400002  31.799999  32.330002  1513400.0  
2024-02-13  31.549999  30.719999  30.950001  1483100.0  
2024-02-14  31.830000  31.170000  31.700001   921800.0  
2024-02-15  32.240002  31.750000  32.070000  1479700.0  
2024-02-16  31.750000  30.969999  30.969999  1053922.0  

[5 rows x 6310 columns]

In [22]:
df_c.tail()

,A,AA,AAL,AAP,AAPL,AB,ABBV,ABR,ABT,ACGL,...,YY,ZBH,ZBRA,ZD,ZG,ZION,ZTO,ZTS,ZUMZ,ZWS
Date,,,,,,,,,,,,,,,,,,,,,
2024-02-12,132.309998,27.900000,14.93,66.050003,187.149994,34.160000,173.070007,12.67,112.529999,83.099998,...,33.810001,125.849998,259.399994,68.519997,53.669998,40.810200,17.459999,196.690002,19.000000,32.330002
2024-02-13,129.759995,25.850000,14.61,64.430000,185.039993,33.439999,173.289993,12.21,111.339996,84.059998,...,31.850000,123.160004,245.800003,64.480003,52.060001,38.930000,17.400000,183.490005,17.870001,30.950001
2024-02-14,132.240005,26.790001,14.86,64.260002,184.149994,33.560001,174.419998,12.74,112.059998,84.870003,...,33.099998,121.839996,252.960007,65.709999,56.029999,40.419998,17.920000,184.080002,18.000000,31.700001
2024-02-15,134.750000,27.389999,14.90,64.650002,183.860001,33.709999,176.589996,13.12,113.639999,87.400002,...,33.570000,124.400002,283.750000,67.699997,55.830002,41.520000,18.129999,189.649994,18.480000,32.070000
2024-02-16,134.839996,27.400000,14.64,64.489998,182.309998,32.610001,177.490005,13.99,114.010002,86.110001,...,33.139999,124.709999,275.459991,66.790001,52.750000,40.639999,18.480000,188.389999,18.309999,30.969999
